In [1]:
# import libraries

## the below give an error - package not found.
#sc.install_pypi_package("pandas==0.25.1")
#sc.install_pypi_package("matplotlib", "https://pypi.org/simple")
#sc.install_pypi_package("scikit-learn")

from pyspark.sql import SparkSession
from pyspark.context import SparkContext as sc
from pyspark.sql.types import IntegerType, LongType, TimestampType
from pyspark import SparkContext
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

import datetime
#import matplotlib.pyplot as plt
#import pandas as pd

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1628680054971_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Load and Clean Dataset

In [4]:
# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
df = spark.read.json(event_data)
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

In [5]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [6]:
# drop null values and empty user IDs

df_valid = df.dropna(how="any")
df_valid = df_valid.select("userId").dropDuplicates().sort("userId")
df_valid = df.filter(df["userId"] != "")
df_valid.describe()
df_valid.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

# Feature Engineering

### Define Churn

Next, we define churn - this in terms of cancellation confirmation and downgrade-related events. Even if dowgrade does not necessarily mean the customer would definitely leave, it is an indicator of potentially decreasing customer satisfaction, lower revenue for the streaming service and also an indicator that the customer might change their mind if they're shown some love and attention. So these are the events we want to be able to identify. 

In [8]:
def is_churn_event(x):
    '''Flags churn events. Returns 1 for churn, 0 for no-churn.'''

    if x in ["Cancellation Confirmation", "Downgrade", "Submit Downgrade"]:
        return 1
    else:
        return 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Define Happy and Unhappy User Behaviours

The Page column includes other user behaviours that are essential for the analysis - I split them into positive/ happy user events and negative/ unhappy events. 

In [9]:
def is_happy_event(x):
    '''Flags happy events. Returns 1 for happy events, 0 otherwise.'''

    if x in [
            "NextSong", "Thumbs Up", "Add Friend", "Add to Playlist", "Upgrade",
            "Submit Upgrade"
    ]:
        return 1
    else:
        return 0


def is_unhappy_event(x):
    '''Flags unhappy events. Returns 1 for unhappy events, 0 otherwise.'''

    if x in [
            "Thumbs Down",
            "Error",
            "Cancel",
    ]:
        return 1
    else:
        return 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
## define UDFs

churn_event = udf(is_churn_event, IntegerType())
happy_event = udf(is_happy_event, IntegerType())
unhappy_event = udf(is_unhappy_event, IntegerType())

get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour)
get_day = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).day)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
## Add Churn/ Happy / Unhappy events to dataframe
## Add Hour / Day / Interaction time features to dataframe
df_valid =  df_valid.withColumn("churn", churn_event("page"))\
            .withColumn("happyEvent", happy_event("page"))\
            .withColumn("unhappyEvent", unhappy_event("page"))\
            .withColumn("hour", get_hour("ts").cast('int'))\
            .withColumn("day", get_day("ts").cast('int'))\
            .withColumn('interactionTime', from_unixtime(col('ts').cast(LongType())/1000).cast(TimestampType()))

# Flag rows for churn / happy / unhappy events.
user_window = Window\
                .partitionBy("userId")\
                .orderBy(desc("ts"))\
                .rangeBetween(Window.unboundedPreceding, 0)

df_valid = df_valid\
            .withColumn("churn", max("churn").over(user_window))\
            .withColumn("happyEvent", max("happyEvent").over(user_window))\
            .withColumn("unhappyEvent", max("unhappyEvent").over(user_window))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_valid.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist=None, auth='Cancelled', firstName='Frank', gender='M', itemInSession=138, lastName='Warren', length=None, level='free', location='Findlay, OH', method='GET', page='Cancellation Confirmation', registration=1535470939000, sessionId=25840, song=None, status=200, ts=1542149985000, userAgent='Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0', userId='1000280', churn=1, happyEvent=0, unhappyEvent=0, hour=22, day=13, interactionTime=datetime.datetime(2018, 11, 13, 22, 59, 45))]
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 59462)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.Re

In [25]:
# number of songs played
songs_count_df = df_valid.select(['userId', 'page',]).where(df.page == 'NextSong') \
                                .groupby('userId') \
                                .agg({'page': 'count'}) \
                                .withColumnRenamed('count(page)', 'songsCount')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
#count number of sessions

sessions_count_df = df_valid.select(['userId', 'sessionId'])\
                            .groupby('userId')\
                            .agg({'sessionId': 'count'})\
                            .withColumnRenamed('count(sessionId)', 'sessionsCount')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#membership period
membership_days_df = df_valid.select(['userId', 'day'])\
                            .groupby('userId')\
                            .agg(max('day'))\
                            .withColumnRenamed('max(day)', 'membershipDays')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#happy events count
happy_events_count_df = df_valid.select(['userId', 'happyEvent',])\
                                .groupby('userId') \
                                .agg(sum('happyEvent'))\
                                .withColumnRenamed('sum(happyEvent)','happyEventCount')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
unhappy_events_count_df = df_valid.select(['userId', 'unhappyEvent',])\
                                .groupby('userId') \
                                .agg(sum('unhappyEvent'))\
                                .withColumnRenamed('sum(unhappyEvent)','unhappyEventCount')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
churn_event_df = df_valid.select(['userId', 'churn',])\
                                .groupby('userId')\
                                .agg(max('churn'))\
                                .withColumnRenamed('max(churn)', 'label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
data = songs_count_df.join(sessions_count_df, 'userId', 'outer') \
                     .join(membership_days_df, 'userId', 'outer')\
                     .join(happy_events_count_df, 'userId', 'outer')\
                     .join(unhappy_events_count_df, 'userId', 'outer')\
                     .join(churn_event_df, 'userId', 'outer')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
data.take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(userId='1000280', songsCount=1022, sessionsCount=1317, membershipDays=26, happyEventCount=1314, unhappyEventCount=1316, label=1), Row(userId='1002185', songsCount=1778, sessionsCount=2080, membershipDays=28, happyEventCount=2080, unhappyEventCount=1968, label=1), Row(userId='1017805', songsCount=250, sessionsCount=320, membershipDays=27, happyEventCount=318, unhappyEventCount=149, label=1)]

In [33]:
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: string (nullable = true)
 |-- songsCount: long (nullable = true)
 |-- sessionsCount: long (nullable = true)
 |-- membershipDays: integer (nullable = true)
 |-- happyEventCount: long (nullable = true)
 |-- unhappyEventCount: long (nullable = true)
 |-- label: integer (nullable = true)

# Modeling

In [ ]:
train, test = data.randomSplit([0.7, 0.3], seed=27)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
features = ['songsCount' , 'sessionsCount', 
            'membershipDays' , 'happyEventCount', 
            'unhappyEventCount' 
           ]

classifiers = [ RandomForestClassifier(featuresCol = 'features', labelCol = 'label'), 
                LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10), 
                GBTClassifier(maxIter=10)
              ]
classifier_names = [ "Random Forest Classifier", 
                     "Logistic Regression", 
                     "Gradient-Boosted Tree Classifier"
                   ]

In [ ]:
def build_model_pipeline (classifier, train): 
    
    '''Input is a classifier and data for training
    Returns fitted model. 
    '''
    
    vectorAssembler = VectorAssembler(inputCols=features, \
                                      outputCol="unscaled_features"
                                     )
    standardScaler = StandardScaler(inputCol="unscaled_features", 
                                    outputCol="features"
                                   )
    stages = [vectorAssembler, standardScaler, classifier]
    pipeline = Pipeline(stages=stages)
    model = pipeline.fit(train)
    
    return model

# Evaluation

In [ ]:
def evaluate_model (model, train, test):
    
    '''Takes as input a fitted model, train data and test data.
    Returns evaluation results: 
    f1 score on train data
    ROC AUC metric on train data
    f1 score on test data
    ROC AUC metric on test data
    '''
    
    # instantiate evaluators 
    binary_evaluator = BinaryClassificationEvaluator(labelCol='label')
    multi_evaluator = MulticlassClassificationEvaluator(labelCol='label')
    
    # make predictions  
    predictions_train = model.transform(train)
    predictions_test = model.transform(test)
    
    f1_train = multi_evaluator.evaluate(predictions_train, {multi_evaluator.metricName: 'f1'})
    roc_train = binary_evaluator.evaluate(predictions_train, {binary_evaluator.metricName: 'areaUnderROC'})
    
    f1_test = multi_evaluator.evaluate(predictions_test, {multi_evaluator.metricName: 'f1'})
    roc_test = binary_evaluator.evaluate(predictions_test, {binary_evaluator.metricName: 'areaUnderROC'})
    
    results = [ f1_train, roc_train, f1_test, roc_test ]
    return results


    

In [ ]:
all_results = dict()

for classifier, name in zip(classifiers, classifier_names):
    
    model = build_model_pipeline(classifier, train)
    result = evaluate_model(model, train, test)
    all_results[name] = result
    


In [ ]:
best_classifier =  LogisticRegression(maxIter=10)

vectorAssembler = VectorAssembler(inputCols=features, \
                                      outputCol="unscaled_features"
                                     )
standardScaler = StandardScaler(inputCol="unscaled_features", 
                                    outputCol="features"
                                   )
stages = [vectorAssembler, standardScaler, best_classifier]
pipeline = Pipeline(stages=stages)
    

paramGrid = ParamGridBuilder() \
            .addGrid(best_classifier.regParam, [0.01, 0.1,]) \
            .addGrid(best_classifier.elasticNetParam, [0.0, 0.5, 1.0])\
            .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3
                          )

In [ ]:
cv_model = crossval.fit(train)
predictions = cv_model.transform(test)
binary_evaluator = BinaryClassificationEvaluator(labelCol='label')
multi_evaluator = MulticlassClassificationEvaluator(labelCol='label')
print("Area Under ROC", binary_evaluator.evaluate(predictions))
print("F1", multi_evaluator.evaluate(predictions))